In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
import tqdm as notebook_tqdm

In [18]:
class BatsmanState(TypedDict):
    runs: int
    balls: int
    fours: int
    sixes: int
    strike_rate: float
    balls_per_boundry: float
    boundary_percentage: float  
    summary: str

In [30]:
def calculate_strike_rate(state: BatsmanState) -> BatsmanState:
    state["strike_rate"] = (state["runs"] / state["balls"])*100
    return {'strike_rate': state['strike_rate']}


In [20]:
def calculate_bpb(state: BatsmanState) -> BatsmanState:
    state['balls_per_boundry'] = state['balls'] / (state['fours'] + state['sixes'])
    return {"balls_per_boundry": state['balls_per_boundry']}    

In [21]:
def calculate_boundary(state: BatsmanState) -> BatsmanState:
    state['boundary_percentage'] = (((state['fours']* 4) + (state['sixes']* 6)) / state['runs'])*100
    return {"boundary_percentage": state['boundary_percentage'] }

In [27]:
def summary(state: BatsmanState):
    summary =f"""Strike Rate: {state['strike_rate']:.2f}, Balls per boundary: {state['balls_per_boundry']:.2f}, Boundary percentage: {state['boundary_percentage']:.2f}"""  
    state['summary'] = summary  
    return {'summary': summary}

In [33]:
graph = StateGraph(BatsmanState)
graph.add_node('calculate_strike_rate', calculate_strike_rate)
graph.add_node('calculate_bpb', calculate_bpb)
graph.add_node('calculate_boundary', calculate_boundary)
graph.add_node('summary', summary)
graph.add_edge(START, 'calculate_strike_rate')
graph.add_edge(START, 'calculate_bpb')
graph.add_edge(START, 'calculate_boundary')
graph.add_edge('calculate_strike_rate', 'summary')
graph.add_edge('calculate_bpb', 'summary')
graph.add_edge('calculate_boundary', 'summary')
graph.add_edge('summary', END)
workflow =graph.compile()


In [34]:
initial_state = {
    "runs": 98,
    "balls": 50,
    "fours": 6,
    "sixes": 4, 
    
}
final_state = workflow.invoke(initial_state)
print(final_state)

{'runs': 98, 'balls': 50, 'fours': 6, 'sixes': 4, 'strike_rate': 196.0, 'balls_per_boundry': 5.0, 'boundary_percentage': 48.97959183673469, 'summary': 'Strike Rate: 196.00, Balls per boundary: 5.00, Boundary percentage: 48.98'}
